In [3]:
import pandas as pd
import pyodbc

# Configuración de la conexión a la base de datos SQL Server
server = '192.168.1.136\MSSQLSERVERDEV'
database = 'URG_Churn_Model'
username = 'ddsusrBC'
password = 'ddsmay10'
driver = '{ODBC Driver 17 for SQL Server}'  # Asegúrate de tener el controlador correcto instalado


# Establecer la cadena de conexión a la base de datos
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}"

# Establecer la conexión con la base de datos
conn = pyodbc.connect(conn_str)

# Crear un cursor para ejecutar consultas SQL
cursor = conn.cursor()

# Nombre de la tabla en la base de datos donde se importará el archivo de Excel
table_name = 'Train'

# Eliminar la tabla si ya existe (opcional)
query = f"SELECT * FROM  {table_name}"

df = pd.read_sql(query, conn)
conn.close()

#print("¡Archivo de Excel importado correctamente en la base de datos!")


C:\Users\jcalabozo\AppData\Local\Temp\ipykernel_13236\2136256932.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304337 entries, 0 to 304336
Data columns (total 39 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   NumeroDeAfiliado                 304337 non-null  int64 
 1   Secuencia                        304337 non-null  object
 2   CodigoDeSucursal                 304337 non-null  object
 3   ApellidoAfiliado                 304337 non-null  object
 4   AnioIndicador                    304337 non-null  int64 
 5   MesIndicador                     304337 non-null  object
 6   TipoAfiliacion                   304337 non-null  object
 7   DescripcionDeSexo                303647 non-null  object
 8   EstadoMesIndicadores             304337 non-null  object
 9   Convenio                         304337 non-null  object
 10  DescripcionDeTipoDePago          304314 non-null  object
 11  DescripcionDeInstitucionDeCobro  294823 non-null  object
 12  UsoServicioAlgun

In [9]:
df['DescripcionDeTipoDePago']=df['DescripcionDeTipoDePago'].fillna('N/A')


In [10]:
df['DescripcionDeTipoDePago'].unique()

array(['Convenio', 'Débito tarjeta', 'Ventanilla', 'Débito bancario',
       'Déb. Recibo Sueldo', 'N/A'], dtype=object)

In [11]:
df['DescripcionDeInstitucionDeCobro']=df['DescripcionDeInstitucionDeCobro'].fillna('N/A')


In [12]:
df['DescripcionDeInstitucionDeCobro'].unique()

array(['Convenio', 'TARJETA NARANJA- CORDOBA', 'N/A', 'NARANJA  AL 8%',
       'COELSA', 'MASTERCARD', 'VISA', 'CABAL', 'CORDOBESA',
       'CMR FALABELLA ROSARIO', 'ACH S.A. - CORDOBA', 'COELSA - CORDOBA',
       'VISA - CORDOBA', 'MASTERCARD - CORDOBA', 'NARANJA',
       'AMERICAN EXPRESS', 'CABAL - CORDOBA', 'AMERICAN EXPRESS- CORDOBA',
       'BBVA Francés - Rosario', 'TARJETA GRUPAR CORDOBA',
       'NATIVA NACION - CORDOBA', 'Empleados  Municipalidad de  Rosario',
       'DINERS', 'NATIVA NACION', 'BBVA Francés - Córdoba', 'SEP',
       'DASPU', 'DINERS - CORDOBA', 'UEPC', 'EFICAZ',
       'CMR FALABELLA CORDOBA', 'Mutual Ley 5110',
       'Asoc Mutual Argentina Nueva', 'MACRO EX SUQUIA',
       'O.Social UNR (Jub.-Pens)', 'MUTUAL CONSEJO DE EDUC VOL II',
       'CMR FALABELLA', 'O.Social UNR (Directos)'], dtype=object)

In [15]:
df['DescripcionDeSexo']=df['DescripcionDeSexo'].fillna('Indefinido')

In [16]:
df['DescripcionDeSexo'].unique()

array(['Femenino', 'Masculino', 'Indefinido'], dtype=object)

In [18]:
# retrieve all labels and store in a list
columnas = df.columns.tolist()

# iterate over the list to print all unique values of each column in the dataframe
for i in columnas:
    print(i,':',df[i].unique())

NumeroDeAfiliado : [ 680824  680836  680842 ... 1135626 1135639 1135659]
Secuencia : ['1' '0' '3' '2' '5' '4' '6' '7' '38' '9' '8' '10' '11' '14' '15' '17'
 '12' '25' '16' '13' '20' '24' '32' '18' '30' '35' '19' '23' '33' '22'
 '34' '21' '28' '29' '31' '40' '41' '42' '36' '26' '44' '39' '27' '43'
 '37' '46' '45']
CodigoDeSucursal : ['2' '1']
ApellidoAfiliado : ['LENCINAS                      ' 'PRATO                         '
 'VAZQUEZ                       ' ... 'ROTILI MASTROGIUSEPPE         '
 'CZAJKOWSKI                    ' 'GIGENA RUARTE                 ']
AnioIndicador : [2021 2009 2011 2017 2018 2010 2019 2008 2016 2012 2015 2020 2014 2013
 2007 2006]
MesIndicador : ['4' '9' '1' '6' '5' '11' '12' '8' '10' '3' '7' '2']
TipoAfiliacion : ['Voluntario' 'Individual']
DescripcionDeSexo : ['Femenino' 'Masculino' 'Indefinido']
EstadoMesIndicadores : ['Activo' 'Suspendido']
Convenio : ['SUOEM' 'UEPC' 'SinConvenio' 'AMR MUTUAL AFILIADOS'
 'OBRA SOCIAL UNR - ACTIVOS' 'IPAM SIN COSEGURO' '